This notebook should be launched from a session of jupyter notebook that was launched from a DB2 command window
To do this, run an administrator DB2 command window as an administrator and type 'jupyter notebook'

Set up the enviornment:

In [ ]:
import sys,os,os.path
os.environ['IBM_DB_HOME']='C:\Program Files\IBM\SQLLIB'
!pip install ipython-sql
!pip install ibm_db 
!pip3 install ifxpy
!pip3 install JayDeBeApi
!pip install ibm_db_sa

Restart the Kernel if this is your first time installing the above. The next steps will fail unless you do this.

Import the modules

In [1]:
import ibm_db
import IfxPy
import pandas as pd
import sqlalchemy
%load_ext sql

Source the file with connection information

In [2]:
# Define filename for passwords
filename = 'ifx_and_db2_variables.py'
# source the file
%run $filename

## Informix Connection

Connect to the database. Change the values to match your environment. For connection to a local host, use 'localhost' for the host name.

In [3]:
ConStr = "SERVER="+ifxserver+";DATABASE="+ifx_DB+";HOST="+ifx_Host+";SERVICE="+ifx_Port+";UID="+ifx_User+";PWD="+ifx_PW+";DB_LOCALE=en_US.utf8;"

try:
    conn = IfxPy.connect( ConStr, "", "")
except Exception as e:
    print ('ERROR: Connect failed')
    print ( e )
    quit()

## Db2 Connection

In [4]:
%sql db2+ibm_db://$db2_User:$db2_PW@$db2_Host:$db2_Port/$db2_DB

'Connected: ecrooks@sample'

## Example #1
Find rows from two days in the past to 7 days in the future from a given, static date

### Db2

In [ ]:
%%sql ex_1_db2 << select *
from sales
where ts_sales_date between date('2006-03-31') - 2 days and date('2006-03-31') + 7 days
order by ts_sales_date
with ur

In [ ]:
ex_1_db2

### Informix

In [ ]:
# Select records
sql = """
select *
from sales
and ts_sales_date::DATETIME year to day between DATE('03/31/2006') - 2 UNITS DAY and DATE('03/31/2006') + 7 UNITS DAY
order by ts_sales_date
"""

stmt = IfxPy.exec_immediate(conn, sql)
display(stmt)
assoc = IfxPy.fetch_assoc(stmt)

ls=[]
while assoc != False:
    ls.append(assoc)
    assoc = IfxPy.fetch_assoc(stmt)



ifx_ex_1=pd.DataFrame(ls)
display(ifx_ex_1.head())
display(ifx_ex_1.tail())

## Example #2 
Find what the date was 36 hours ago

### Db2

In [5]:
%%sql ex_2_db2 << select date(current timestamp - 36 hours) 
from sysibm.sysdummy1

 * db2+ibm_db://ecrooks:***@localhost:50000/sample
Done.
Returning data to local variable ex_2_db2


In [6]:
ex_2_db2

1
2019-03-13


### Informix

In [7]:
# Select records
sql = """
select (current - 36 units hour)::DATETIME YEAR to DAY from sysmaster:sysdual 
"""

stmt = IfxPy.exec_immediate(conn, sql)
display(stmt)
assoc = IfxPy.fetch_assoc(stmt)

ls=[]
while assoc != False:
    ls.append(assoc)
    assoc = IfxPy.fetch_assoc(stmt)



ifx_ex_2=pd.DataFrame(ls)
display(ifx_ex_2)


,
0,2019-03-13


## Example #3
Find the day of the week one month ago

### Db2

In [8]:
%%sql ex_3_db2 << select dayname(current timestamp - 1 month) weekday 
from sysibm.sysdummy1

 * db2+ibm_db://ecrooks:***@localhost:50000/sample
Done.
Returning data to local variable ex_3_db2


In [9]:
ex_3_db2

weekday
Thursday


### Informix

In [10]:
# Select records
sql = """
select case WEEKDAY(current - 1 units month) 
  when 0 then 'Sunday'
  when 1 then 'Monday'
  when 2 then 'Tuesday'
  when 3 then 'Wednesday'
  when 4 then 'Thursday'
  when 5 then 'Friday'
  when 6 then 'Saturday' end as weekday
from sysmaster:sysdual 
"""

stmt = IfxPy.exec_immediate(conn, sql)
display(stmt)
assoc = IfxPy.fetch_assoc(stmt)

ls=[]
while assoc != False:
    ls.append(assoc)
    assoc = IfxPy.fetch_assoc(stmt)



ifx_ex_3=pd.DataFrame(ls)
display(ifx_ex_3)

,weekday
0,Thursday


## Example #4
Find the exact time 450 seconds from now

### Db2

In [11]:
%%sql ex_4_db2 << select current timestamp + 450 seconds time 
from sysibm.sysdummy1

 * db2+ibm_db://ecrooks:***@localhost:50000/sample
Done.
Returning data to local variable ex_4_db2


In [12]:
ex_4_db2

TIME
2019-03-14 19:14:26.794000


### Informix

In [13]:
# Select records
sql = """
select current + 450 units second from sysmaster:sysdual 
"""

stmt = IfxPy.exec_immediate(conn, sql)
display(stmt)
assoc = IfxPy.fetch_assoc(stmt)

ls=[]
while assoc != False:
    ls.append(assoc)
    assoc = IfxPy.fetch_assoc(stmt)



ifx_ex_4=pd.DataFrame(ls)
display(ifx_ex_4)

,
0,2019-03-14 20:59:25


In [14]:
# Free up memory used by informix result and then stmt too
IfxPy.free_result(stmt)
IfxPy.free_stmt (stmt)

IfxPy.close(conn)

print ("Done")


Done


## Format for password file:
```python
ifx_Host='example.example.com'
ifx_DB='sample'
ifx_Port='50001'
ifx_User='username'
ifx_PW='pw'
ifxserver='reporting_tcp'

db2_Host='example.example.com'
db2_DB='sample'
db2_Port='50000'
db2_User='user'
db2_PW='pw'
```